# nvImageCodecs with cuCIM (only Linux)

In [ ]:
import os
from matplotlib import pyplot as plt
import cupy as cp

In [ ]:
from nvidia import nvimgcodecs
decoder = nvimgcodecs.Decoder()
encoder = nvimgcodecs.Encoder()
resources_dir = "../../resources"

In [ ]:
nv_img4k = decoder.decode(resources_dir + "/base/4k_lossless.jp2")

Convert to cupy ndarray

In [ ]:
cp_img4k = cp.asarray(nv_img4k)

In [ ]:
cp_img_4k_single_channel = cp_img4k[:, :, 1]

In [ ]:
from cucim.skimage.filters import gaussian
cp_img4k_blurred = gaussian(cp_img_4k_single_channel, sigma=15)
image = cp.asnumpy(cp_img4k_blurred)
plt.imshow(image, cmap='gray')

In [ ]:
from cucim.skimage.filters import threshold_otsu
# determine threshold
threshold = threshold_otsu(cp_img4k_blurred)

# binarize image by apply the threshold
cp_binary_gpu = cp_img4k_blurred > threshold

image = cp.asnumpy(cp_binary_gpu)
plt.imshow(image, cmap='gray')

In [ ]:
from cucim.skimage.morphology import binary_erosion, disk

eroded_gpu = binary_erosion(cp_binary_gpu, selem=disk(2))
image = cp.asnumpy(eroded_gpu)
plt.imshow(image, cmap='gray')


In [ ]:
cp_img4k_blurred_color = gaussian(cp_img4k, sigma=35)
image = cp.asnumpy(cp_img4k_blurred_color)
plt.imshow(image)

Please notice that cuCIM returned blurred color image as a HWC and in float

In [ ]:
print(cp_img4k_blurred_color.__cuda_array_interface__)
cp_img4k_blurred_color = cp_img4k_blurred_color*255
cp_img4k_blurred_color_int = cp_img4k_blurred_color.astype("u1")
print(cp_img4k_blurred_color_int.__cuda_array_interface__)

In [ ]:
nv_img = nvimgcodecs.as_image(cp_img4k_blurred_color_int)

In [ ]:
encoder.encode("blurred.jpg", nv_img)

In [ ]:
import cv2
image = cv2.imread("blurred.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
